In [31]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import CSVLogger
import pandas as pd
import numpy as np

In [2]:
dataset = pd.read_csv('../datas/RGB_color_labels.csv') # read the dataset using pandas library
dataset = pd.get_dummies(dataset, columns=['label']) # adds 11 new columns that one hot encodes the class of the color

In [3]:
train_dataset = dataset.sample(frac=0.8, random_state=8) #train_dataset = 80% of total dataset  
#random_state = any int value means every time when you run your program you will get the same output for train and test dataset, random_state is None by default which means every time when you run your program you will get different output because of splitting between train and test varies within 
test_dataset = dataset.drop(train_dataset.index) #remove train_dataset from dataframe to get test_dataset


# Separates the dataset(that will be the input) and the labels(that will be the class of the input values). This part is for training.
train_labels = pd.DataFrame()
for x in ['label_Red', 'label_Green', 'label_Blue', 'label_Yellow', 'label_Orange', 'label_Pink', 'label_Purple', 'label_Brown', 'label_Grey', 'label_Black', 'label_White']:
    train_labels[x] = train_dataset.pop(x)


# Separates the dataset(that will be the input) and the labels(that will be the class of the input values). This part is for testing.
test_labels = pd.DataFrame()
for x in ['label_Red', 'label_Green', 'label_Blue', 'label_Yellow', 'label_Orange', 'label_Pink', 'label_Purple', 'label_Brown', 'label_Grey', 'label_Black', 'label_White']:
    test_labels[x] = test_dataset.pop(x)


# Converts all data into numpy arrays
# Why convert? Because the code gives a lot of errors if you did not input a numpy or tensors array to the neural network.
# I will leave to the reader to research about tensors in the documentation. https://www.tensorflow.org/guide/tensor
train_dataset = np.array(train_dataset)
train_labels = np.array(train_labels)
train_labels = np.argmax(train_labels, axis=1)

test_dataset = np.array(test_dataset)
test_labels = np.array(test_labels)
test_labels = np.argmax(test_labels, axis=1)

In [56]:
class ModelTrainer:
    def __init__(self, model_path) -> None:
        self.model_path = model_path
        self.train_path = f"{self.model_path}/saved_per_train" # The train path of the model
        self.last_trained = os.listdir(self.train_path)[-1] # This gives the latest trained
        self.checkpoint_list = os.listdir(f"{self.train_path}/{self.last_trained}")
        try:
            self.checkpoint_list.remove("logs.csv")
        except:
            pass
        self.least_loss_model = min(self.checkpoint_list, key=lambda loss_val:loss_val[-4:-1])
        self.least_loss_checkpoint_path = f"{self.train_path}/{self.last_trained}/{self.least_loss_model}" # The tensor model of the latest trained
        self.loaded_model = tf.keras.models.load_model(self.least_loss_checkpoint_path) # Loads the model

    def train_model(self, epochs=100, batch_size=1000, save_freq=100):
        self.last_trained = os.listdir(self.train_path)[-1] # This gives the latest trained
        self.checkpoint_list = os.listdir(f"{self.train_path}/{self.last_trained}")
        self.checkpoint_list.remove("logs.csv")
        self.least_loss_model = min(self.checkpoint_list, key=lambda loss_val:loss_val[-4:-1])
        self.least_loss_checkpoint_path = f"{self.train_path}/{self.last_trained}/{self.least_loss_model}" # The tensor model of the latest trained
        self.loaded_model = tf.keras.models.load_model(self.least_loss_checkpoint_path) # Loads the model

        self.last_model_num = os.listdir(f"{self.train_path}")[-1][-1]
        os.makedirs(f"{self.train_path}/train{int(last_model_num) + 1}")
        self.last_train_folder = f"{self.train_path}/train{int(last_model_num) + 1}"

        # Training the model
        checkpoint_path = f"{last_train_folder}" + "/Epoch{epoch:02d}_loss{loss:.2f}"
        csv_logger = CSVLogger(f"{last_train_folder}/logs.csv", separator=',', append=False)

        # Create a callback that saves the model's weights
        cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 verbose=1,
                                                 monitor='accuracy',
                                                 save_freq=save_freq) # if save_freq='epochs' it saves the model per epoch
                                                                # if save_freq=int_type it saves the model per <int_type> of batches

        self.loaded_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
              
        # Train the model with the new callback
        self.loaded_model.fit(train_dataset, 
                train_labels,  
                epochs=epochs,
                batch_size=batch_size,
                validation_data=(test_dataset, test_labels),
                callbacks=[cp_callback, csv_logger], # Pass callback to training
                shuffle=True) 
        # This may generate warnings related to saving the state of the optimizer.
        # These warnings (and similar warnings throughout this notebook)
        # are in place to discourage outdated usage, and can be ignored.

    def make_csv(self, specific_train=None):
        import xlsxwriter

        if not specific_train:
            self.last_model_num = os.listdir(f"{self.train_path}")[-1][-1]
            self.train_folder = f"{self.train_path}/train{int(self.last_model_num)}"
            last_trained_checkpoint_list = os.listdir(self.last_train_folder)
            try:
                last_trained_checkpoint_list.remove('logs.csv')
            except:
                pass
        elif specific_train:
            self.train_folder = f"{self.train_path}/{specific_train}"
            last_trained_checkpoint_list = os.listdir(f"{self.train_path}/{specific_train}")
            try:
                last_trained_checkpoint_list.remove('logs.csv')
            except:
                pass
            


        for model in last_trained_checkpoint_list:
            loaded_model = tf.keras.models.load_model(f"{self.train_folder}/{model}")

            relative_row_idx = 0 # this is for writing bias
            row_idx = 0
            max_row_idx = 0
            column_idx = 0

            workbook = xlsxwriter.Workbook(f"{self.train_folder}/{model}/saved_weights_biases.xlsx")
            worksheet = workbook.add_worksheet()

            for layer in loaded_model.layers:
                for row_weights in layer.get_weights()[0].T: # the reason I transposed the matrix because tensorflow makes weights in transposed position of matrix
                    for weights in row_weights:
                        worksheet.write(row_idx, column_idx, weights)
                        column_idx += 1

                    column_idx += 1
                    worksheet.write(row_idx, column_idx, layer.get_weights()[1].T[relative_row_idx]) # the reason I transposed the matrix because tensorflow makes weights in transposed position of matrix
                    relative_row_idx += 1
                    row_idx += 1
                    column_idx = 0

                relative_row_idx = 0
                row_idx += 2
            workbook.close()    

In [58]:
load_model1 = ModelTrainer("../trained_models/model1")

In [59]:
load_model1.make_csv(specific_train="train2")

In [9]:
print(load_model1.last_trained)

train1
